In [1]:
### load Data 
import tensorflow as tf
import gzip
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import keras as ks
import keras
import numpy as np
import keras.backend as K
from random import random
from random import randint
from numpy import array
from numpy import zeros
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers import TimeDistributed
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import AveragePooling1D
from keras.callbacks import LambdaCallback
from keras import optimizers
from keras.layers.merge import concatenate
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.utils import multi_gpu_model
import multiprocessing
#from eli5.sklearn import PermutationImportance
#from numba import jit
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
#from hyperopt import Trials, STATUS_OK, tpe
#from hyperas import optim
#from hyperas.distributions import choice
import numpy as np
import pickle 
import os
from keras.preprocessing.sequence import TimeseriesGenerator

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8697467438462794785
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15094329791039619669
physical_device_desc: "device: XLA_CPU device"
]


Using TensorFlow backend.


In [2]:

import multiprocessing
#import dask.dataframe as dk
import pandas as pd
import numpy as np
import datetime as dt

#import matplotlib.pyplot as plt
idx=pd.IndexSlice
from sklearn.metrics import make_scorer, r2_score,accuracy_score,precision_score
from sklearn.externals import joblib
import os
import gc
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from tqdm import tqdm
import inspect 


/home/sr4376/miniconda3/envs/testCheckv3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:

multiprocessing.cpu_count()



48

In [4]:

mainFrame=pd.read_csv('/beegfs/sr4376/Finance Data/CNN-LSTM Data/NormalizedAllDataAndTargetSharpe.csv',parse_dates=["date"])


In [5]:
mainFrame.set_index(['entityID','date'],inplace=True)
#mainFrame=mainFrame20052018
mainFrame.dropna(axis=0,inplace=True)
mainFrame.sort_index(inplace=True)
#mainFrame=mainFrame[~(mainFrame.index.get_level_values(1)>'2012')]


In [6]:
mainFrame.dropna(axis=0,inplace=True)
mainFrame.sort_index(inplace=True)

In [7]:
mainFrame.drop(columns=mainFrame.columns[175:203],inplace=True)

In [8]:

targets=mainFrame['targetForSharpe']
features = mainFrame.iloc[:,:-8]


In [23]:
targets=mainFrame['ztargetMedian5']

In [9]:
featureImportance=pd.read_csv('D:\checkFeatures.csv')

In [10]:
featureImportance.set_index('Unnamed: 0',inplace=True)

In [11]:
featureImportance.nlargest(150,columns='0').index.get_level_values(0)

Index(['crossStd120', 'crossStd160', 'crossStd220', 'crossStd240',
       'crossStd180', 'crossStd100', 'crossStd200', 'volatility_atr_x',
       'crossStd140', 'price240',
       ...
       'momentum_wr_xcross', 'trend_macd_signal_x', 'momentum_wr_x',
       'trend_kst_xcross', 'momentum_ao_x', 'volatility_dcl_xcross',
       'volume_obv_x', 'price40', 'return7cross', 'return19'],
      dtype='object', name='Unnamed: 0', length=150)

In [12]:
features=features.loc[:,featureImportance.nlargest(100,columns='0').index.get_level_values(0)]

In [13]:


class DataGenerator(keras.utils.Sequence):
    def __init__(self, feature,target, length, batch_size=128):
        entityIDs=feature.index.get_level_values(0).unique()
        self.tgs = list()
        self.indexObject = pd.DataFrame([])
        for i in tqdm(entityIDs):
         
            featuredf = feature[feature.index.get_level_values(0) == i]
            targetdf = target[target.index.get_level_values(0) == i]

            tmpIDX = pd.DataFrame([featuredf.index.get_level_values(0).values,featuredf.index.get_level_values(1).values]).T
            if self.indexObject.empty:
                self.indexObject = tmpIDX.iloc[length:]
            else:
                self.indexObject = self.indexObject.append(tmpIDX.iloc[length:])
            if len(targetdf)>length:
                self.tgs.append(TimeseriesGenerator(featuredf.values,targetdf.values,length,batch_size =batch_size))
        self.len = sum([len(tg) for tg in self.tgs])
        self.idx_i = list()
        self.idx_j = list()

        for i, tg in enumerate(self.tgs):
            self.idx_i.extend(list(range(len(tg))))
            self.idx_j.extend([i]*len(tg))    
        #print ( self.idx_i,  self.idx_j)

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        return self.tgs[self.idx_j[index]][self.idx_i[index]]


In [14]:
features

Unnamed: 0           crossStd120  crossStd160  crossStd220  crossStd240  \
entityID date                                                             
1034     2007-03-01    -0.570219    -0.823187    -0.467222     0.664990   
         2007-03-02    -0.572254    -0.826444    -0.457100     0.951922   
         2007-03-05    -0.576295    -0.827778    -0.467256     0.923943   
         2007-03-06    -0.570456    -0.813596    -0.849633    -0.023207   
         2007-03-07    -0.572158    -0.811341    -0.839926    -0.189590   
...                          ...          ...          ...          ...   
296318   2015-06-18     1.245677     0.009199     0.391921    -0.196737   
         2015-06-19     1.070683     0.045367     0.379149     0.164005   
         2015-06-22     1.454468    -0.059594     0.483926     0.321188   
         2015-06-23     1.696421    -0.179802     0.476902     0.429468   
         2015-06-24     1.712907    -0.237267     0.246639     0.355254   

Unnamed: 0           crossStd180  crossStd100  crossStd200  volatility_atr_x  \
entityID date                                                                  
1034     2007-03-01    -0.882222    -0.716819    -0.808114         -1.509487   
         2007-03-02    -0.880660    -0.724401    -0.808288         -1.178735   
         2007-03-05    -0.889035    -0.729095    -0.811433         -1.280377   
         2007-03-06    -0.889297    -0.722382    -0.803159         -1.181582   
         2007-03-07    -0.887652    -0.722108    -0.801651         -1.116762   
...                          ...          ...          ...               ...   
296318   2015-06-18     0.284253     0.566662    -0.334788         -1.014274   
         2015-06-19     0.440173     0.523445    -0.223256         -1.086186   
         2015-06-22     0.468035     0.868782    -0.058789         -1.201015   
         2015-06-23     0.257172     0.788427    -0.194545         -1.299266   
         2015-06-24     0.285387     0.712145    -0.344485         -1.400290   

Unnamed: 0           crossStd140  price240  ...   price80  crossStd16  \
entityID date                               ...                         
1034     2007-03-01    -0.835545  1.115381  ... -0.366971   -0.269859   
         2007-03-02    -0.839499  0.924579  ... -0.675602   -0.268175   
         2007-03-05    -0.846750  0.913169  ... -0.772324   -0.268561   
         2007-03-06    -0.834050  0.922681  ... -0.343498   -0.269671   
         2007-03-07    -0.833558  0.751781  ... -0.441518   -0.268705   
...                          ...       ...  ...       ...         ...   
296318   2015-06-18     0.040891 -1.158990  ... -1.722372   -0.444328   
         2015-06-19    -0.066659 -1.169022  ... -1.659232   -0.470532   
         2015-06-22    -0.077641 -1.167338  ... -1.731273   -0.447631   
         2015-06-23     0.039690 -1.189902  ... -1.672453   -0.451785   
         2015-06-24    -0.081649 -1.250384  ... -1.675201   -0.549656   

Unnamed: 0            return4   return5  volatility_dchi_x  momentum_tsi_x  \
entityID date                                                                
1034     2007-03-01 -1.125645 -1.163187          -0.346410        2.164435   
         2007-03-02 -2.061060 -1.972516          -0.346410        1.942531   
         2007-03-05 -1.051231 -1.981813          -0.346410        1.735325   
         2007-03-06 -0.417034 -0.482202          -0.346410        1.585055   
         2007-03-07 -1.166322 -0.781970          -0.346410        1.413958   
...                       ...       ...                ...             ...   
296318   2015-06-18 -0.814077 -1.064222          -0.233224        0.246631   
         2015-06-19 -0.453941 -0.420047          -0.223607        0.187194   
         2015-06-22 -0.332348 -0.332876          -0.213643        0.135294   
         2015-06-23 -0.263860 -0.397089          -0.213643        0.084415   
         2015-06-24  0.152692 -0.383478          -0.213643        0.032530   

Unnamed: 0        

In [19]:

datesCheck=features.index.get_level_values(1).unique().sort_values()
allDates = features.index.get_level_values(1)
gc.collect()
predictionPeriod=1
LSTMLength=21
yearsBack=np.arange(1,2)
featureLength=features.shape[1]
for jj in yearsBack:
    years=np.arange(2007,2015)
    for ii in years:
        print(ii)
        lowYear=ii-jj
        TrainStartdate =  datesCheck[np.argwhere((datesCheck == datesCheck[datesCheck > str(ii-jj)].min())==True)-LSTMLength]        
        
        TrainDataMask  = (allDates>=TrainStartdate[0][0])  & (allDates<=str(ii))
        
        TrainData=features[TrainDataMask]
        Traintarget = targets[TrainDataMask]
        TrainSet = DataGenerator(TrainData,Traintarget,LSTMLength,1)
        trainData=list()
        trainTarget=list()
        
        for kk in tqdm(range(0,len(TrainSet))):
            trainData.append(TrainSet[kk][0].reshape((LSTMLength,featureLength,1)))
        trainData = np.array(trainData)
        
        with gzip.open('/beegfs/sr4376/Finance Data/LSTM/yearsBack/CNNFeatureYear' + str(ii) +'lookBackYear' + str(jj) + 'LSTMWindow' + str(LSTMLength) + 'NumberOfFeatures' + str(features.shape[1]) + '.pklz', 'wb') as handle:
            pickle.dump(trainData, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        
        trainData=[]
        
        
        for kk in tqdm(range(0,len(TrainSet))):
            trainTarget.append(TrainSet[kk][1].reshape((1)))
        trainTarget = np.array(trainTarget) 
               

        with gzip.open('/beegfs/sr4376/Finance Data/LSTM/yearsBack/CNNTargetYear' + str(ii) +'lookBackYear' +str(jj) + 'LSTMWindow' + str(LSTMLength) + 'NumberOfFeatures' + str(features.shape[1]) +  '.pklz', 'wb') as handle:
            pickle.dump(trainTarget, handle, protocol=pickle.HIGHEST_PROTOCOL)
         
        
        trainTarget=[]
        
        with open('/beegfs/sr4376/Finance Data/LSTM/yearsBack/indexObjectYear' + str(ii) +'lookBackYear' +str(jj) + 'LSTMWindow' + str(LSTMLength) + 'NumberOfFeatures' + str(features.shape[1]) +  '.csv', 'wb') as handle:
            TrainSet.indexObject.to_csv('/beegfs/sr4376/Finance Data/LSTM/yearsBack/indexObjectYear' + str(ii) +'lookBackYear' +str(jj) + 'LSTMWindow' + str(LSTMLength) + 'NumberOfFeatures' + str(features.shape[1]) +  '.csv')
        
        

/home/sr4376/miniconda3/envs/testCheckv3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Support for multi-dimensional indexing (e.g. `index[:, None]`) on an Index is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  del sys.path[0]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

2007


2008


100%|██████████| 586134/586134 [00:14<00:00, 39375.70it/s]


2009


...

100%|██████████| 679161/679161 [00:17<00:00, 39223.59it/s]


2013


100%|██████████| 681305/681305 [00:17<00:00, 39627.27it/s]


2014


100%|██████████| 701837/701837 [00:17<00:00, 40353.11it/s]


In [20]:

datesCheck=features.index.get_level_values(1).unique().sort_values()
allDates = features.index.get_level_values(1)
gc.collect()
predictionPeriod=1
LSTMLength=21
yearsBack=np.arange(1,2)
featureLength=features.shape[1]
for jj in yearsBack:
    years=np.arange(2007,2015)
    for ii in years:
        print(ii)
        lowYear=ii-jj
        TrainStartdate =  datesCheck[np.argwhere((datesCheck == datesCheck[datesCheck > str(ii-jj)].min())==True)-LSTMLength]        
        
        TrainDataMask  = (allDates>=TrainStartdate[0][0])  & (allDates<=str(ii))
        
        TrainData=features[TrainDataMask]
        Traintarget = targets[TrainDataMask]
        TrainSet = DataGenerator(TrainData,Traintarget,LSTMLength,1)
        trainData=list()
        trainTarget=list()
        
        for kk in tqdm(range(0,len(TrainSet))):
            trainData.append(TrainSet[kk][0].reshape((LSTMLength,featureLength)))
        trainData = np.array(trainData)
        
        with gzip.open('/beegfs/sr4376/Finance Data/LSTM/yearsBack/FeatureYear' + str(ii) +'lookBackYear' + str(jj) + 'LSTMWindow' + str(LSTMLength) + 'NumberOfFeatures' + str(features.shape[1]) + '.pklz', 'wb') as handle:
            pickle.dump(trainData, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        
        trainData=[]
        
        
        for kk in tqdm(range(0,len(TrainSet))):
            trainTarget.append(TrainSet[kk][1].reshape((1)))
        trainTarget = np.array(trainTarget) 
               

        with gzip.open('/beegfs/sr4376/Finance Data/LSTM/yearsBack/TargetYear' + str(ii) +'lookBackYear' +str(jj) + 'LSTMWindow' + str(LSTMLength) + 'NumberOfFeatures' + str(features.shape[1]) +  '.pklz', 'wb') as handle:
            pickle.dump(trainTarget, handle, protocol=pickle.HIGHEST_PROTOCOL)
         
        
        trainTarget=[]
        
        with open('/beegfs/sr4376/Finance Data/LSTM/yearsBack/indexObjectYear' + str(ii) +'lookBackYear' +str(jj) + 'LSTMWindow' + str(LSTMLength) + 'NumberOfFeatures' + str(features.shape[1]) +  '.csv', 'wb') as handle:
            TrainSet.indexObject.to_csv('/beegfs/sr4376/Finance Data/LSTM/yearsBack/indexObjectYear' + str(ii) +'lookBackYear' +str(jj) + 'LSTMWindow' + str(LSTMLength) + 'NumberOfFeatures' + str(features.shape[1]) +  '.csv')
        
        

/home/sr4376/miniconda3/envs/testCheckv3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Support for multi-dimensional indexing (e.g. `index[:, None]`) on an Index is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  del sys.path[0]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

2007
2008



100%|██████████| 586134/586134 [00:14<00:00, 40380.53it/s]


2009


100%|██████████| 616867/616867 [00:15<00:00, 40205.55it/s]


...

100%|██████████| 679161/679161 [00:16<00:00, 40051.98it/s]


2013


100%|██████████| 681305/681305 [00:17<00:00, 39970.99it/s]


2014


100%|██████████| 701837/701837 [00:17<00:00, 39855.10it/s]
